In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.models import load_model
from keras.losses import mean_absolute_error
import tensorflow as tf
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from keras.utils import plot_model

Using TensorFlow backend.
C:\Users\AKSHAT GUPTA\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\AKSHAT GUPTA\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\AKSHAT GUPTA\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\AKSHAT GUPTA\Anaconda3\lib\site-packages\tensorflo

In [2]:
train_path = 'D:/Flipkart GRID - Object Localization (Medium Complexity Round)/augument_data/training_images'
npy_img_dir = 'D:/Flipkart GRID - Object Localization (Medium Complexity Round)\\augument_data\\training_images_batches'
architecture_plot_save_dir = 'D:/Flipkart GRID - Object Localization (Medium Complexity Round)/'

img_width, img_height = 299., 299.
rx = 2.1404
ry = 1.61

In [3]:
##### img_width, img_height = 640, 480
base_dir = 'D:/Flipkart GRID - Object Localization (Medium Complexity Round)/'
# train_data_dir = "X:/train-scene classification/data/train/"
# validation_data_dir = "X:/train-scene classification/data/valid/"
# test_data_dir = "X:/train-scene classification/data/test/"
epochs = 1000
# Save the model according to the conditions  
chkpt_dir = "D:/Flipkart GRID - Object Localization (Medium Complexity Round)/chkpts/inceptionresnetv2_l1loss_chkpt.h5" # Change for every new training session 

In [8]:
# base_model = ResNet50(weights = "imagenet", include_top=False, input_shape = (img_height, img_width, 3), classes = 4)
base_model = InceptionResNetV2(weights = "imagenet", include_top=False, input_shape = (int(img_height), int(img_width), 3), classes = 4)

In [7]:
# base_model.save('inceptionresnetv2.h5')
# base_model = load_model(os.path.join(base_dir,'base_models', 'inceptionresnetv2.h5'))

In [9]:
# plot_model(base_model, to_file= os.path.join(architecture_plot_save_dir, 'inceptiontresnetv2.png'))
base_model.summary()

Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_204 (Conv2D)             (None, 149, 149, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_204 (BatchN (None, 149, 149, 32) 96          conv2d_204[0][0]                 
__________________________________________________________________________________________________
activation_204 (Activation)     (None, 149, 149, 32) 0           batch_normalization_204[0][0]    
________________________________________________________________________________

__________________________________________________________________________________________________
batch_normalization_304 (BatchN (None, 17, 17, 192)  576         conv2d_304[0][0]                 
__________________________________________________________________________________________________
batch_normalization_307 (BatchN (None, 17, 17, 192)  576         conv2d_307[0][0]                 
__________________________________________________________________________________________________
activation_304 (Activation)     (None, 17, 17, 192)  0           batch_normalization_304[0][0]    
__________________________________________________________________________________________________
activation_307 (Activation)     (None, 17, 17, 192)  0           batch_normalization_307[0][0]    
__________________________________________________________________________________________________
block17_7_mixed (Concatenate)   (None, 17, 17, 384)  0           activation_304[0][0]             
          

activation_373 (Activation)     (None, 8, 8, 224)    0           batch_normalization_373[0][0]    
__________________________________________________________________________________________________
conv2d_371 (Conv2D)             (None, 8, 8, 192)    399360      block8_1_ac[0][0]                
__________________________________________________________________________________________________
conv2d_374 (Conv2D)             (None, 8, 8, 256)    172032      activation_373[0][0]             
__________________________________________________________________________________________________
batch_normalization_371 (BatchN (None, 8, 8, 192)    576         conv2d_371[0][0]                 
__________________________________________________________________________________________________
batch_normalization_374 (BatchN (None, 8, 8, 256)    768         conv2d_374[0][0]                 
__________________________________________________________________________________________________
activation

In [5]:
for layer in base_model.layers[:-3]:
    layer.trainable = False
base_model.summary()

Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
________________________________________________________________________________

__________________________________________________________________________________________________
conv2d_125 (Conv2D)             (None, 17, 17, 192)  208896      block17_12_ac[0][0]              
__________________________________________________________________________________________________
conv2d_128 (Conv2D)             (None, 17, 17, 192)  215040      activation_127[0][0]             
__________________________________________________________________________________________________
batch_normalization_125 (BatchN (None, 17, 17, 192)  576         conv2d_125[0][0]                 
__________________________________________________________________________________________________
batch_normalization_128 (BatchN (None, 17, 17, 192)  576         conv2d_128[0][0]                 
__________________________________________________________________________________________________
activation_125 (Activation)     (None, 17, 17, 192)  0           batch_normalization_125[0][0]    
__________

In [13]:
base_model.input

<tf.Tensor 'input_2:0' shape=(?, 299, 299, 3) dtype=float32>

In [21]:
# for layer in base_model.layers:
#     layer.trainable = False
x = base_model.output
# x = Flatten()(x)
# x = Dense(2048, activation="relu")(x)
# x = Dropout(0.7)(x)
# x = Dense(32, activation="relu")(x)
# x = Dropout(0.2)(x)
# x = Dense(16, activation="relu")(x)
# x = Dropout(0.0)(x)
# x = Dense(64, activation="relu")(x)
# x = Dropout(0.0)(x)
# x = Dense(1024, activation="relu")(x)
# x = Dropout(0.5)(x)
# x = Dense(512, activation="relu")(x)
# x = Dropout(0.3)(x)
# predictions = Dense(6, activation="softmax")(x)
# predictions = Dense(4, activation="relu")(x)
x

<tf.Tensor 'conv_7b_ac_1/Relu:0' shape=(?, 8, 8, 1536) dtype=float32>

In [22]:
Flatten()(x)

<tf.Tensor 'flatten_4/Reshape:0' shape=(?, ?) dtype=float32>

In [20]:
x.shape
x

<tf.Tensor 'flatten_3/Reshape:0' shape=(?, ?) dtype=float32>

In [7]:
model = Model(input = base_model.input, output = predictions)
# for layer in model.layers[:-3]:
#     layer.trainable = False
model.summary()

C:\Users\AKSHAT GUPTA\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

__________________________________________________________________________________________________
activation_148 (Activation)     (None, 17, 17, 192)  0           batch_normalization_148[0][0]    
__________________________________________________________________________________________________
block17_18_mixed (Concatenate)  (None, 17, 17, 384)  0           activation_145[0][0]             
                                                                 activation_148[0][0]             
__________________________________________________________________________________________________
block17_18_conv (Conv2D)        (None, 17, 17, 1088) 418880      block17_18_mixed[0][0]           
__________________________________________________________________________________________________
block17_18 (Lambda)             (None, 17, 17, 1088) 0           block17_17_ac[0][0]              
                                                                 block17_18_conv[0][0]            
__________

In [10]:
def mIoU(labels,predictions):
#     print(labels.shape)
#     print(predictions.shape)
#     print(labels[0])
#     print(predictions[0])
    pred_x1=predictions[:,0]#*img_width
    pred_x2=predictions[:,1]#*img_width
    pred_y1=predictions[:,2]#*img_height
    pred_y2=predictions[:,3]#*img_height
    label_x1=labels[:,0]#*img_width
    label_x2=labels[:,1]#*img_width
    label_y1=labels[:,2]#*img_height
    label_y2=labels[:,3]#*img_height
    int_x1 = tf.maximum(pred_x1, label_x1)
    int_x2 = tf.minimum(pred_x2, label_x2)
    int_y1 = tf.maximum(pred_y1, label_y1)
    int_y2 = tf.minimum(pred_y2, label_y2)
    tensor_type = pred_x1.dtype
    pred_area = tf.multiply(tf.maximum(tf.cast(0.0, tensor_type), pred_x2 - pred_x1), tf.maximum(tf.cast(0.0, tensor_type), pred_y2 - pred_y1))
    label_area = tf.multiply(tf.maximum(tf.cast(0.0, tensor_type), label_x2 - label_x1), tf.maximum(tf.cast(0.0, tensor_type), label_y2 - label_y1))
    int_area = tf.multiply(tf.maximum(tf.cast(0.0, tensor_type), int_x2 - int_x1), tf.maximum(tf.cast(0.0, tensor_type), int_y2 - int_y1))
    union_area = pred_area + label_area - int_area
    IoU = int_area / union_area
    return tf.reduce_mean(IoU)

In [11]:
# IoU Metric Test
LABEL = [[0.0, 1.0, 0.0, 1.0], 
        [2.0, 3.0, 2.0, 3.0]]
PRED = [[0.5, 1.5, 0.5, 1.5],
        [2.0, 3.0, 2.0, 3.0]]
LABEL = tf.constant(np.array(LABEL))
PRED = tf.constant(np.array(PRED))
print(mIoU(LABEL, PRED))
with tf.Session() as sess:
    val = sess.run(mIoU(LABEL, PRED))
    print(val)

Tensor("Mean:0", shape=(), dtype=float64)
0.5714285714285714


#### Load best model weights from here, if required.

In [13]:
# model.load_weights(chkpt_dir)

In [14]:
model.compile(loss = mean_absolute_error, optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8), metrics=[mIoU])

In [15]:
checkpoint = ModelCheckpoint(chkpt_dir, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=40, verbose=1, mode='auto')

In [16]:
train_df = pd.read_csv(os.path.join(base_dir, 'augument_data', 'augmented_train_data.csv'))
val_df = pd.read_csv(os.path.join(os.path.join(base_dir, 'augument_data', 'augmented_val_data.csv')))

In [17]:
l = []
for im in val_df.image_name:
    img = (cv2.imread(os.path.join(train_path, im))) ### INTRIDUCING NORMALIZATION.
    img = cv2.resize(img, (int(img_width), int(img_height)))
    img = (img / 255.).astype(np.float32)
    l.append(np.expand_dims(img, axis = 0))
x_val = np.vstack(l)
print("x_val done.")
y_val = val_df.iloc[:,1:].values.astype(np.float32)
y_val[:, 0] = y_val[:, 0] # / img_width
y_val[:, 1] = y_val[:, 1] # / img_width
y_val[:, 2] = y_val[:, 2] # / img_height
y_val[:, 3] = y_val[:, 3] # / img_height
print(y_val[0])
print(y_val.dtype)
print(x_val.shape)


x_val done.
[ 78. 578.  78. 317.]
float32
(2000, 299, 299, 3)


In [18]:
print(val_df.image_name[0])
# model.load_weights(chkpt_dir)

[h_v]1475157534672DSC07249.png


In [19]:
ram_batch_size = 1000
total_number_images = 68000
print(total_number_images // ram_batch_size)

68


In [20]:
def show_img_with_rect(img, lab):
    x1 = int(lab[0]/2.1404)
    x2 = int(lab[1]/2.1404)
    y1 = int(lab[2]/1.61)
    y2 = int(lab[3]/1.61) #
    cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),3)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [21]:
# show_img_with_rect(x_val[654], y_val[654])

In [22]:
ctr = 0

for epoch in range(epochs):
    print(epoch)
    print("--------------------------------------------------------------")
    for i in range(total_number_images // ram_batch_size):
        print('batch {}/67 (epoch {}):'.format(i, epoch))
        l = []
#         for im in train_df.image_name[i * ram_batch_size: (i + 1) * ram_batch_size]:
#             img = (cv2.imread(os.path.join(train_path, im)) / 255.).astype(np.float32) ### INTRIDUCING NORMALIZATION.
#             l.append(np.expand_dims(img, axis = 0))
        x_train = np.load(os.path.join(npy_img_dir, str(i) + '.npy'))
        y_train = train_df[i * ram_batch_size: (i + 1) * ram_batch_size].iloc[:,1:].values
#         y_train[:, 0] = y_train[:, 0] # / img_width
#         y_train[:, 1] = y_train[:, 1] # / img_width
#         y_train[:, 2] = y_train[:, 2] # / img_height
#         y_train[:, 3] = y_train[:, 3] # / img_height
#         plt.figure(i)
#         show_img_with_rect(x_train[0], y_train[0])
        model.fit(x=x_train, y=y_train, batch_size=32, epochs=1, verbose=1, callbacks=[checkpoint], validation_split=0.0, validation_data=(x_val, y_val), shuffle=True)
#         if i == 0:
#             pred = model.predict(np.expand_dims(x_train[0], axis=0))
#             X1 = int(pred[0][0]/rx)
#             X2 = int(pred[0][1]/rx)
#             Y1 = int(pred[0][2]/ry)
#             Y2 = int(pred[0][3]/ry)
#             cv2.rectangle(x_train[0],(X1,Y1),(X2,Y2),(0,255,0),3)
#             plt.figure(ctr)
#             ctr += 1
#             plt.imshow(cv2.cvtColor(x_train[0], cv2.COLOR_BGR2RGB))

0
--------------------------------------------------------------
batch 0/67 (epoch 0):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 33s 33ms/step - loss: 114.8381 - mIoU: 0.3176 - val_loss: 132.5201 - val_mIoU: 0.1298

Epoch 00001: val_loss improved from inf to 132.52014, saving model to X:/Flipkart GRID - Object Localization (Medium Complexity Round)/chkpts/inceptionresnetv2_l1loss_chkpt.h5
batch 1/67 (epoch 0):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 44.7508 - mIoU: 0.6091 - val_loss: 107.9439 - val_mIoU: 0.1905

Epoch 00001: val_loss improved from 132.52014 to 107.94392, saving model to X:/Flipkart GRID - Object Localization (Medium Complexity Round)/chkpts/inceptionresnetv2_l1loss_chkpt.h5
batch 2/67 (epoch 0):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 35.4761

Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 19.6441 - mIoU: 0.7886 - val_loss: 25.3207 - val_mIoU: 0.7434

Epoch 00001: val_loss improved from 25.77218 to 25.32071, saving model to X:/Flipkart GRID - Object Localization (Medium Complexity Round)/chkpts/inceptionresnetv2_l1loss_chkpt.h5
batch 28/67 (epoch 0):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 20.9332 - mIoU: 0.7810 - val_loss: 38.1646 - val_mIoU: 0.6159

Epoch 00001: val_loss did not improve from 25.32071
batch 29/67 (epoch 0):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 18.2793 - mIoU: 0.8038 - val_loss: 35.0657 - val_mIoU: 0.6423

Epoch 00001: val_loss did not improve from 25.32071
batch 30/67 (epoch 0):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============

1000/1000 [==============================] - 27s 27ms/step - loss: 15.7712 - mIoU: 0.8287 - val_loss: 26.5681 - val_mIoU: 0.7299

Epoch 00001: val_loss did not improve from 24.35294
batch 58/67 (epoch 0):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 17.5428 - mIoU: 0.8123 - val_loss: 26.6277 - val_mIoU: 0.7245

Epoch 00001: val_loss did not improve from 24.35294
batch 59/67 (epoch 0):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 15.8917 - mIoU: 0.8276 - val_loss: 35.3323 - val_mIoU: 0.6474

Epoch 00001: val_loss did not improve from 24.35294
batch 60/67 (epoch 0):
Train on 1000 samples, validate on 2000 samples
Epoch 1/1
1000/1000 [==============================] - 27s 27ms/step - loss: 16.5947 - mIoU: 0.8208 - val_loss: 32.6203 - val_mIoU: 0.6616

Epoch 00001: val_loss did not improve from 24.35294
batch 61/67 (epoch 0):
Train 

KeyboardInterrupt: 

In [ ]:
model.save('inceptionresnetv2_l1loss.h5')

In [ ]:
model.load_weights(chkpt_dir)

In [ ]:
for i in range(0, 2000, 100):
    pred = model.predict(np.expand_dims(x_val[i], axis=0))
    a1 = int(pred[0][0]/rx)
    a2 = int(pred[0][1]/rx)
    b1 = int(pred[0][2]/ry)
    b2 = int(pred[0][3]/ry)
    test_img = x_val[i]
    print(test_img.shape)
    plt.figure(i)
    cv2.rectangle(test_img,(a1,b1),(a2,b2),(0,255,0),3)
    plt.imshow(cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB))

In [ ]:
test_img = x_val[7]
test_img = np.expand_dims(test_img, axis=0)
pred = model.predict(test_img)
print(pred.shape)
a1 = int(pred[0][0]/rx)
a2 = int(pred[0][1]/rx)
b1 = int(pred[0][2]/ry)
b2 = int(pred[0][3]/ry)
test_img = test_img[0]
cv2.rectangle(test_img,(a1,b1),(a2,b2),(0,255,0),3)
plt.imshow(cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB))

In [ ]:
model.save('kuchcha_inception_resnet_v2.h5')

In [ ]:
model.load_weights(chkpt_dir)
LABEL = y_val
PRED = model.predict(x_val)
with tf.Session() as sess:
    val = sess.run(mIoU(LABEL, PRED))
    print(val)

In [ ]:
test_df = pd.read_csv(os.path.join('X:/Flipkart GRID - Object Localization (Medium Complexity Round)/orig_data', 'test.csv'))

In [ ]:
print(type(test_df))

In [ ]:
for i, im in enumerate(test_df.image_name):
    img = cv2.imread(os.path.join('X:/Flipkart GRID - Object Localization (Medium Complexity Round)/orig_data/train', im))